<a href="https://colab.research.google.com/github/martinezger/rcnn-mvc/blob/main/vehicle_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dependencies and imports
This notebook use tensorflow-gpu 1.15.5 this will trigger a pip error at the end of the cell execution, please ignore the error because those dependencies won't affect us.

In [ ]:
!pip uninstall -y -q keras-nightly
!pip install -q keras==2.0.8 tensorflow-gpu==1.15.5 h5py==2.10.0 mrcnn-colab==2.1

In [4]:
import os
import sys
import json
import datetime
import numpy as np
import skimage.draw
import urllib
from mrcnn.config import Config
from mrcnn import model as modellib, utils

Using TensorFlow backend.


# Download Files and create Root Folder

This setup should work out of the box, in case you want to move to a custom path just change the ROOT_DIR.

In [ ]:
ROOT_DIR = os.path.abspath("test")
DEFAULT_LOGS_DIR = os.path.join(ROOT_DIR, "logs")

DATA_SET_PATH = os.path.join(ROOT_DIR, "rcnn-mvc-main/dataset")
COCO_WEIGHTS_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")

COCO_RCNN_URL = "https://github.com/matterport/Mask_RCNN/releases/download/v2.0/mask_rcnn_coco.h5"
DATA_SET_URL = "https://github.com/martinezger/rcnn-mvc/tree/main/dataset"
MRCNN_REPO_URL = "https://github.com/martinezger/rcnn-mvc/archive/refs/heads/main.zip"

if not os.path.exists(ROOT_DIR):
  print("create project root folder")
  os.mkdir(ROOT_DIR)

!wget "https://github.com/matterport/Mask_RCNN/releases/download/v2.0/mask_rcnn_coco.h5" -P {ROOT_DIR}
!wget "https://github.com/martinezger/rcnn-mvc/archive/refs/heads/main.zip" -P {ROOT_DIR}
!unzip -qq {os.path.join(ROOT_DIR, "main.zip")} "rcnn-mvc-main/dataset/*" -d {ROOT_DIR}
!rm {os.path.join(ROOT_DIR, "main.zip")}

# Model Setup


In [7]:

############################################################
#  Configurations
############################################################

class CustomCarConfig(Config):
    """Configuration for training on the toy  dataset.
    Derives from the base Config class and overrides some values.
    """
    # Give the configuration a recognizable name
    NAME = "customCar"

    # We use a GPU with 12GB memory, which can fit two images.
    # Adjust down if you use a smaller GPU.
    IMAGES_PER_GPU = 2

    # Number of classes (including background)
    NUM_CLASSES = 1 + 3  # Background + cars + bus + truck

    # Number of training steps per epoch
    STEPS_PER_EPOCH = 14

    # Skip detections with < 90% confidence
    DETECTION_MIN_CONFIDENCE = 0.9


############################################################
#  Dataset
############################################################

class CustomCarDataset(utils.Dataset):

    def load_car(self, dataset_dir, subset):
        """Load a subset of the Balloon dataset.
        dataset_dir: Root directory of the dataset.
        subset: Subset to load: train or val
        """
        # Add classes. We have only one class to add.
        self.add_class("customCar", 1, "car")
        self.add_class("customCar", 2, "truck")
        self.add_class("customCar", 3, "bus")

        # Train or validation dataset?
        assert subset in ["train", "val"]
        dataset_dir = os.path.join(dataset_dir, subset)

        annotations = json.load(open(os.path.join(dataset_dir, "via_region_data.json")))
        annotations = list(annotations.values())  # don't need the dict keys

        # The VIA tool saves images in the JSON even if they don't have any
        # annotations. Skip unannotated images.
        annotations = [a for a in annotations if a['regions']]

        for a in annotations:
            if type(a['regions']) is dict:
                polygons = [r['shape_attributes'] for r in a['regions'].values()]
            else:
                polygons = [r['shape_attributes'] for r in a['regions']]

            image_path = os.path.join(dataset_dir, a['filename'])
            image = skimage.io.imread(image_path)
            height, width = image.shape[:2]
            name = [r["region_attributes"]["name"] for r in a["regions"]]
            name_dict = {"car": 1, "truck": 2, "bus": 3}
            name_id = [name_dict[a] for a in name]

            self.add_image(
                "customCar",
                image_id=a['filename'],  # use file name as a unique image id
                path=image_path,
                class_id=name_id,
                width=width,
                height=height,
                polygons=polygons)

    def load_mask(self, image_id):
        """Generate instance masks for an image.
       Returns:
        masks: A bool array of shape [height, width, instance count] with
            one mask per instance.
        class_ids: a 1D array of class IDs of the instance masks.
        """
        # If not a balloon dataset image, delegate to parent class.
        image_info = self.image_info[image_id]
        if image_info["source"] != "customCar":
            return super(self.__class__, self).load_mask(image_id)

        name_id = image_info['class_id']

        # Convert polygons to a bitmap mask of shape
        # [height, width, instance_count]
        info = self.image_info[image_id]
        mask = np.zeros([info["height"], info["width"], len(info["polygons"])],
                        dtype=np.uint8)

        class_ids = np.array(name_id, dtype=np.int32)

        for i, p in enumerate(info["polygons"]):
            # Get indexes of pixels inside the polygon and set them to 1
            rr, cc = skimage.draw.polygon(p['all_points_y'], p['all_points_x'])
            mask[rr, cc, i] = 1

        # Return mask, and array of class IDs of each instance. Since we have
        # one class ID only, we return an array of 1s
        return mask.astype(np.bool), class_ids

    def image_reference(self, image_id):
        """Return the path of the image."""
        info = self.image_info[image_id]
        if info["source"] == "customCar":
            return info["path"]
        else:
            super(self.__class__, self).image_reference(image_id)


def train(model, config):
    """Train the model."""
    # Training dataset.
    dataset_train = CustomCarDataset()
    dataset_train.load_car(DATA_SET_PATH, "train")
    dataset_train.prepare()

    # Validation dataset
    dataset_val = CustomCarDataset()
    dataset_val.load_car(DATA_SET_PATH, "val")
    dataset_val.prepare()

    # *** This training schedule is an example. Update to your needs ***
    # Since we're using a very small dataset, and starting from
    # COCO trained weights, we don't need to train too long. Also,
    # no need to train all layers, just the heads should do it.
    print("Training network heads")
    model.train(dataset_train, dataset_val,
                learning_rate=config.LEARNING_RATE,
                epochs=30,
                layers='heads')


def color_splash(image, mask):
    """Apply color splash effect.
    image: RGB image [height, width, 3]
    mask: instance segmentation mask [height, width, instance count]
    Returns result image.
    """
    # Make a grayscale copy of the image. The grayscale copy still
    # has 3 RGB channels, though.
    gray = skimage.color.gray2rgb(skimage.color.rgb2gray(image)) * 255
    # Copy color pixels from the original color image where mask is set
    if mask.shape[-1] > 0:
        # We're treating all instances as one, so collapse the mask into one layer
        mask = (np.sum(mask, -1, keepdims=True) >= 1)
        splash = np.where(mask, image, gray).astype(np.uint8)
    else:
        splash = gray.astype(np.uint8)
    return splash


def run_train(command="train", dataset=DATA_SET_PATH, weights="coco", logs=DEFAULT_LOGS_DIR):
    # Validate arguments
    if command == "train":
        assert dataset, "Argument --dataset is required for training"

    print("Weights: ", weights)
    print("Dataset: ", dataset)
    print("Logs: ", logs)

    # Configurations
    if command == "train":
        config = CustomCarConfig()
    else:
        class InferenceConfig(CustomCarConfig):
            # Set batch size to 1 since we'll be running inference on
            # one image at a time. Batch size = GPU_COUNT * IMAGES_PER_GPU
            GPU_COUNT = 1
            IMAGES_PER_GPU = 1

        config = InferenceConfig()
    config.display()

    # Create model
    if command == "train":
        model = modellib.MaskRCNN(mode="training", config=config,
                                  model_dir=logs)
    else:
        model = modellib.MaskRCNN(mode="inference", config=config,
                                  model_dir=logs)

    # Select weights file to load
    if weights.lower() == "coco":
        weights_path = COCO_WEIGHTS_PATH
        # Download weights file
        if not os.path.exists(weights_path):
            utils.download_trained_weights(weights_path)
    elif weights.lower() == "last":
        # Find last trained weights
        weights_path = model.find_last()
    elif weights.lower() == "imagenet":
        # Start from ImageNet trained weights
        weights_path = model.get_imagenet_weights()
    else:
        weights_path = weights

    # Load weights
    print("Loading weights ", weights_path)
    if weights.lower() == "coco":
        # Exclude the last layers because they require a matching
        # number of classes
        model.load_weights(weights_path, by_name=True, exclude=[
            "mrcnn_class_logits", "mrcnn_bbox_fc",
            "mrcnn_bbox", "mrcnn_mask"])
    else:
        model.load_weights(weights_path, by_name=True)

    # Train or evaluate
    if command == "train":
        train(model, config)

    else:
        print("'{}' is not recognized. "
              "Use 'train' or 'splash'".format(command))


# Run Training
After hit start the training will start. Base on some testing I made could take between 30 to 60 minutes to finish a traning.

In [ ]:
import warnings
warnings.simplefilter(action="ignore", category=FutureWarning)
run_train()